In [1]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn import naive_bayes #import GaussianNB, MultinomialNB
from sklearn.linear_model import SGDClassifier # simulate the behavior of logistic regression using SGDClassifier(loss='log')
from sklearn.metrics import accuracy_score,balanced_accuracy_score,average_precision_score, classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from collections import Counter
from itertools import product

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
# Load the trained vectorizer
with open('tfidf_vectorizer_train_split_33k.pkl', 'rb') as f:
    loaded_vectorizer = pickle.load(f)
len(loaded_vectorizer.get_feature_names_out())

train_df = pd.read_csv('cfpb_train.csv')
test_df = pd.read_csv('cfpb_test.csv')
dev_df = pd.read_csv('cfpb_dev.csv')

C:\Users\JYM\AppData\Local\Temp/ipykernel_22612/2270551363.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('cfpb_train.csv')
C:\Users\JYM\AppData\Local\Temp/ipykernel_22612/2270551363.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dev_df = pd.read_csv('cfpb_dev.csv')


In [4]:
# some cleaning just ot make sure
train_df['Consumer complaint narrative'] = train_df['Consumer complaint narrative'].fillna('').astype(str)
test_df['Consumer complaint narrative'] = test_df['Consumer complaint narrative'].fillna('').astype(str)
dev_df['Consumer complaint narrative'] = dev_df['Consumer complaint narrative'].fillna('').astype(str)

train_df['debt_collection'] = (train_df['Product'] == 'Debt collection').astype(int)
test_df['debt_collection'] = (test_df['Product'] == 'Debt collection').astype(int)
dev_df['debt_collection'] = (dev_df['Product'] == 'Debt collection').astype(int)

In [5]:
%%time
X_test = loaded_vectorizer.transform(test_df['Consumer complaint narrative'])
y_test = test_df['debt_collection']

X_dev = loaded_vectorizer.transform(dev_df['Consumer complaint narrative'])
y_dev = dev_df['debt_collection']

Wall time: 56.7 s


### Grid Search Model

In [7]:
# # %%time
# # # Define the parameters for exploration
# # # First Run
# # # param_grid = {
# # #     'sample_size': [50000, 100000, 150000, 200000], # Example sizes, please adjust according to your needs
# # #     'chi2_features': [10000, 20000, 30000], # Example feature numbers, please adjust according to your needs
# # #     'clf__alpha': [1e-4, 1e-3, 1e-2],
# # #     'clf__loss': ['hinge', 'log_loss', 'squared_hinge', ],
# # #     'clf__penalty': ['elasticnet'],
# # #     'clf__n_iter_no_change': [3],
# # #     'clf__early_stopping': [True, False],
# # #     'clf__learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
# # #     'clf__eta0': [0.01, 0.1]
# # # }

# #Second Run
# param_grid = {
#     'sample_size': [100000, 200000, 300000, 400000, 500000], # Example sizes, please adjust according to your needs
#     'chi2_features': [20000, 25000, 30000], # Example feature numbers, please adjust according to your needs
#     'clf__alpha': [1e-4],
#     'clf__loss': ['hinge', 'squared_hinge'], # 'squared_hinge' performs slightly better, but that is non-linear SVM
#     'clf__penalty': ['elasticnet'],
#     'clf__n_iter_no_change': [3],
#     'clf__early_stopping': [False], # Does not show huge difference
#     'clf__learning_rate': ['constant', 'adaptive'],
#     'clf__eta0': [0.01]
# }

# # #Third Run
# # param_grid = {
# #     'sample_size': [200000, 250000, 300000, 400000], # Example sizes, please adjust according to your needs
# #     'chi2_features': [20000, 22500, 25000], # Example feature numbers, please adjust according to your needs
# #     'clf__alpha': [1e-4],
# #     'clf__loss': ['log_loss'], # 'squared_hinge' performs slightly better, but that is non-linear SVM
# #     'clf__penalty': ['elasticnet'],
# #     'clf__n_iter_no_change': [3],
# #     'clf__early_stopping': [False], # Does not show huge difference
# #     'clf__learning_rate': ['constant'],
# #     'clf__eta0': [0.01]
# # }




# # Grid search
# # Here I will be defining a function to encapsulate the model training for different parameters
# # For example, the 'sample_size' and 'chi2_features' parameters are not a part of the SGDClassifier
# def grid_search_model(param_grid):
#     best_params = None
#     best_score = 0
#     prev_score = 0
    
#     for sample_size in param_grid['sample_size']:
#         for chi2_features in param_grid['chi2_features']:
#             # Sample and transform the train data
#             train_df_sample = train_df.sample(sample_size)
            
#             X_train = loaded_vectorizer.transform(train_df_sample['Consumer complaint narrative'])
#             y_train = train_df_sample['debt_collection']
            
#             selector = SelectKBest(chi2, k=chi2_features)
#             X_train = selector.fit_transform(X_train, y_train)
#             # Transform dev set with the same selector
#             X_dev_transformed = selector.transform(X_dev)
            
#             oversample = RandomOverSampler(sampling_strategy='minority')
#             X_train, y_train = oversample.fit_resample(X_train, y_train)
            
#             # Shuffle your data
#             X_train, y_train = shuffle(X_train, y_train)
            
#             # The product function is from Python's built-in itertools module. It computes the Cartesian product of input iterables, 
#             # which means it generates all possible combinations of the input iterables' elements.
#             for params in product(
#                 param_grid['clf__alpha'],
#                 param_grid['clf__loss'],
#                 param_grid['clf__penalty'],
#                 param_grid['clf__n_iter_no_change'],
#                 param_grid['clf__early_stopping'],
#                 param_grid['clf__learning_rate'],
#                 param_grid['clf__eta0']
#             ):
                
#                 # Apply the parameters
#                 alpha, loss, penalty, n_iter_no_change, early_stopping, learning_rate, eta0 = params
                
#                 clf = SGDClassifier(
#                     loss=loss, 
#                     penalty=penalty, 
#                     alpha=alpha, 
#                     n_iter_no_change=n_iter_no_change, 
#                     early_stopping=early_stopping, 
#                     learning_rate=learning_rate, 
#                     eta0=eta0, 
#                     random_state=42
#                 )
#                 # Train and score the model
#                 clf.fit(X_train, y_train)
#                 # F1 score: This is the harmonic mean of precision and recall, and it tries to balance these two values. 
#                 # It is especially useful if your classes are unevenly distributed.
#                 predicted = clf.predict(X_dev_transformed)
#                 score = f1_score(y_dev, predicted)
                
#                 if score > best_score:
#                     best_score = score
#                     best_params = {
#                         'sample_size': sample_size,
#                         'chi2_features': chi2_features,
#                         'clf__alpha': alpha,
#                         'clf__loss': loss,
#                         'clf__penalty': penalty,
#                         'clf__n_iter_no_change': n_iter_no_change,
#                         'clf__early_stopping': early_stopping,
#                         'clf__learning_rate': learning_rate,
#                         'clf__eta0': eta0
#                     }
#                     print(f"New best score:{score} using {best_params}")
# #                 else:
# #                     print(f"Currentscore:{score} using {best_params}")

#                 # If the improvement in the score is less than 0.01, stop the iterations
# #                 if abs(score - prev_score) < 0.01:
# #                     return best_params

#                 prev_score = score
                    
#     return best_params

# best_params = grid_search_model(param_grid)
# print(f'Best parameters: {best_params}')

New best score:0.7470619614943179 using {'sample_size': 100000, 'chi2_features': 20000, 'clf__alpha': 0.0001, 'clf__loss': 'hinge', 'clf__penalty': 'elasticnet', 'clf__n_iter_no_change': 3, 'clf__early_stopping': False, 'clf__learning_rate': 'constant', 'clf__eta0': 0.01}
New best score:0.7472328306039226 using {'sample_size': 100000, 'chi2_features': 20000, 'clf__alpha': 0.0001, 'clf__loss': 'hinge', 'clf__penalty': 'elasticnet', 'clf__n_iter_no_change': 3, 'clf__early_stopping': False, 'clf__learning_rate': 'adaptive', 'clf__eta0': 0.01}
New best score:0.7528047984585263 using {'sample_size': 100000, 'chi2_features': 20000, 'clf__alpha': 0.0001, 'clf__loss': 'squared_hinge', 'clf__penalty': 'elasticnet', 'clf__n_iter_no_change': 3, 'clf__early_stopping': False, 'clf__learning_rate': 'constant', 'clf__eta0': 0.01}
New best score:0.7537653896631461 using {'sample_size': 100000, 'chi2_features': 20000, 'clf__alpha': 0.0001, 'clf__loss': 'squared_hinge', 'clf__penalty': 'elasticnet', 'cl

### SVM

In [6]:
%%time
sample_size =400000
chi2_features = 30000
alpha = 0.0001
loss = 'squared_hinge'
penalty=  'elasticnet'
n_iter_no_change = 3
early_stopping = False
learning_rate = 'constant'
eta0 =  0.01


train_df_sample = train_df.sample(sample_size)
            
X_train = loaded_vectorizer.transform(train_df_sample['Consumer complaint narrative'])
y_train = train_df_sample['debt_collection']

selector = SelectKBest(chi2, k=chi2_features)
X_train = selector.fit_transform(X_train, y_train)
# Transform dev set with the same selector
X_dev_transformed = selector.transform(X_dev)

oversample = RandomOverSampler(sampling_strategy='minority')
X_train, y_train = oversample.fit_resample(X_train, y_train)

clf = SGDClassifier(
                    loss=loss, 
                    penalty=penalty, 
                    alpha=alpha, 
                    n_iter_no_change=n_iter_no_change, 
                    early_stopping=early_stopping, 
                    learning_rate=learning_rate, 
                    eta0=eta0, 
                    random_state=42
                )
clf.fit(X_train, y_train)

Wall time: 1min 25s


SGDClassifier(eta0=0.01, learning_rate='constant', loss='squared_hinge',
              n_iter_no_change=3, penalty='elasticnet', random_state=42)

In [8]:
# get the results
y_pred = clf.predict(X_train.toarray())

# Convert lists to arrays for further use
y_pred = np.array(y_pred)

print("Train")
print("accuracy_score",accuracy_score(y_train, y_pred))
print("balanced_accuracy_score",balanced_accuracy_score(y_train, y_pred))
print("average_precision_score",average_precision_score(y_train, y_pred))
print(classification_report(y_train, y_pred))

c = Counter(y_pred)
print("Prediction", c.most_common(2))
c = Counter(y_train)
print("Ground Truth",c.most_common(2))

Train
accuracy_score 0.8938022294707211
balanced_accuracy_score 0.8938022294707211
average_precision_score 0.860248566656397
              precision    recall  f1-score   support

           0       0.87      0.92      0.90    328957
           1       0.91      0.87      0.89    328957

    accuracy                           0.89    657914
   macro avg       0.89      0.89      0.89    657914
weighted avg       0.89      0.89      0.89    657914

Prediction [(0, 345606), (1, 312308)]
Ground Truth [(0, 328957), (1, 328957)]


In [9]:
X_dev = selector.transform(X_dev)

# get the results
y_pred = clf.predict(X_dev.toarray())

# Convert lists to arrays for further use
y_pred = np.array(y_pred)

print("Train")
print("accuracy_score",accuracy_score(y_dev, y_pred))
print("balanced_accuracy_score",balanced_accuracy_score(y_dev, y_pred))
print("average_precision_score",average_precision_score(y_dev, y_pred))
print(classification_report(y_dev, y_pred))

c = Counter(y_pred)
print("Prediction", c.most_common(2))
c = Counter(y_dev)
print("Ground Truth",c.most_common(2))

Train
accuracy_score 0.9057627118644068
balanced_accuracy_score 0.8845267432436247
average_precision_score 0.6124947368370434
              precision    recall  f1-score   support

           0       0.97      0.92      0.94    136036
           1       0.69      0.85      0.76     29164

    accuracy                           0.91    165200
   macro avg       0.83      0.88      0.85    165200
weighted avg       0.92      0.91      0.91    165200

Prediction [(0, 129118), (1, 36082)]
Ground Truth [(0, 136036), (1, 29164)]


svm - Parameter Combinations that will Yield Similar Good Results

New best score:0.7470619614943179 using {'sample_size': 100000, 'chi2_features': 20000, 'clf__alpha': 0.0001, 'clf__loss': 'hinge', 'clf__penalty': 'elasticnet', 'clf__n_iter_no_change': 3, 'clf__early_stopping': False, 'clf__learning_rate': 'constant', 'clf__eta0': 0.01}

**New best score:0.7472328306039226 using {'sample_size': 100000, 'chi2_features': 20000, 'clf__alpha': 0.0001, 'clf__loss': 'hinge', 'clf__penalty': 'elasticnet', 'clf__n_iter_no_change': 3, 'clf__early_stopping': False, 'clf__learning_rate': 'adaptive', 'clf__eta0': 0.01}**

New best score:0.7528047984585263 using {'sample_size': 100000, 'chi2_features': 20000, 'clf__alpha': 0.0001, 'clf__loss': 'squared_hinge', 'clf__penalty': 'elasticnet', 'clf__n_iter_no_change': 3, 'clf__early_stopping': False, 'clf__learning_rate': 'constant', 'clf__eta0': 0.01}

New best score:0.7537653896631461 using {'sample_size': 100000, 'chi2_features': 20000, 'clf__alpha': 0.0001, 'clf__loss': 'squared_hinge', 'clf__penalty': 'elasticnet', 'clf__n_iter_no_change': 3, 'clf__early_stopping': False, 'clf__learning_rate': 'adaptive', 'clf__eta0': 0.01}

New best score:0.7545093625933914 using {'sample_size': 100000, 'chi2_features': 25000, 'clf__alpha': 0.0001, 'clf__loss': 'squared_hinge', 'clf__penalty': 'elasticnet', 'clf__n_iter_no_change': 3, 'clf__early_stopping': False, 'clf__learning_rate': 'constant', 'clf__eta0': 0.01}

New best score:0.754944487073083 using {'sample_size': 100000, 'chi2_features': 25000, 'clf__alpha': 0.0001, 'clf__loss': 'squared_hinge', 'clf__penalty': 'elasticnet', 'clf__n_iter_no_change': 3, 'clf__early_stopping': False, 'clf__learning_rate': 'adaptive', 'clf__eta0': 0.01}

New best score:0.7610322139090667 using {'sample_size': 200000, 'chi2_features': 20000, 'clf__alpha': 0.0001, 'clf__loss': 'squared_hinge', 'clf__penalty': 'elasticnet', 'clf__n_iter_no_change': 3, 'clf__early_stopping': False, 'clf__learning_rate': 'constant', 'clf__eta0': 0.01}

**New best score:0.7613263958996661 using {'sample_size': 200000, 'chi2_features': 20000, 'clf__alpha': 0.0001, 'clf__loss': 'squared_hinge', 'clf__penalty': 'elasticnet', 'clf__n_iter_no_change': 3, 'clf__early_stopping': False, 'clf__learning_rate': 'adaptive', 'clf__eta0': 0.01}**

New best score:0.7615425218120284 using {'sample_size': 200000, 'chi2_features': 25000, 'clf__alpha': 0.0001, 'clf__loss': 'squared_hinge', 'clf__penalty': 'elasticnet', 'clf__n_iter_no_change': 3, 'clf__early_stopping': False, 'clf__learning_rate': 'adaptive', 'clf__eta0': 0.01}

New best score:0.7618870093356324 using {'sample_size': 200000, 'chi2_features': 30000, 'clf__alpha': 0.0001, 'clf__loss': 'squared_hinge', 'clf__penalty': 'elasticnet', 'clf__n_iter_no_change': 3, 'clf__early_stopping': False, 'clf__learning_rate': 'adaptive', 'clf__eta0': 0.01}

New best score:0.7630364881867747 using {'sample_size': 300000, 'chi2_features': 20000, 'clf__alpha': 0.0001, 'clf__loss': 'squared_hinge', 'clf__penalty': 'elasticnet', 'clf__n_iter_no_change': 3, 'clf__early_stopping': False, 'clf__learning_rate': 'adaptive', 'clf__eta0': 0.01}

**New best score:0.765276310064027 using {'sample_size': 400000, 'chi2_features': 30000, 'clf__alpha': 0.0001, 'clf__loss': 'squared_hinge', 'clf__penalty': 'elasticnet', 'clf__n_iter_no_change': 3, 'clf__early_stopping': False, 'clf__learning_rate': 'constant', 'clf__eta0': 0.01}**

Best parameters: {'sample_size': 400000, 'chi2_features': 30000, 'clf__alpha': 0.0001, 'clf__loss': 'squared_hinge', 'clf__penalty': 'elasticnet', 'clf__n_iter_no_change': 3, 'clf__early_stopping': False, 'clf__learning_rate': 'constant', 'clf__eta0': 0.01}

It requires 200,000 samples and about 20,000 features to yield acceptable results.

But best results yield from 400,000 samples with 30,000 features.